In [ ]:
import pandas as pd
import numpy as np
import mplfinance as mpf

df = pd.read_csv(r'D:\Programming\Programming\Visual Studio\Bollinger_Bands\download_data_and_calculate_indicator\calculate_AVWAP_indicator\df2_saved.csv')

df['datetime'] = pd.to_datetime(df['datetime'])
df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time
df.reset_index(drop=True, inplace=True)

df.head() 

df_py = df[['date', 'time', 'datetime', 'open', 'high', 'low', 'close', 'volume']] 


In [ ]:
def calculate_initial_values(df):

    # Ensure df is a copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Set rising and falling conditions
    df.loc[:, 'Py_rising_'] = df['close'] > df['open']
    df.loc[:, 'Py_falling_'] = df['open'] > df['close']

    # Calculate core candle properties
    candle_body = np.round(np.abs(df['close'] - df['open']), 2)
    rising = df['close'] > df['open']
    falling = df['open'] > df['close']
    up_wick = np.round(np.where(rising, df['high'] - df['close'], df['high'] - df['open']), 2)
    dn_wick = np.round(np.where(falling, df['close'] - df['low'], df['open'] - df['low']), 2)

    # Calculate Doji condition
    df.loc[:, 'Py_doji_'] = ((up_wick >= candle_body) | (dn_wick >= candle_body)) & \
                                (df['high'] != df['open']) & \
                                (df['high'] != df['close']) & \
                                (df['low'] != df['open']) & \
                                (df['low'] != df['close'])

    # Reverse Doji condition
    df.loc[:, 'Py_doji_'] = df['Py_doji_'].replace({True: False, False: True})

    # Initialize columns with proper assignment
    df.loc[:, 'Py_c_'] = df['close']
    df.loc[:, 'Py_c1_'] = df['close'].shift(1).fillna(0.0)
    df.loc[:, 'Py_h1_'] = df['high'].shift(1).fillna(0.0)
    df.loc[:, 'Py_l1_'] = df['low'].shift(1).fillna(0.0)
    df.loc[:, 'Py_o_'] = df['open']

    return df

def prepare_data(data):
    df = pd.DataFrame(data)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.sort_values(['date', 'datetime']).reset_index(drop=True)
    return df

def calculate_vwap(df, anchor='Session', column_name = 'VWAP'):
    df = df.copy()
    
    df['Typical_Price'] = (df['high'] + df['low'] + df['close']) / 3
    df['TPV'] = df['Typical_Price'] * df['volume']
    
    # Define the anchor period
    if anchor == 'Session':
        df['anchor'] = df['datetime'].dt.date
    elif anchor == 'Week':
        df['anchor'] = df['datetime'].dt.to_period('W').apply(lambda r: r.start_time)
    elif anchor == 'Month':
        df['anchor'] = df['datetime'].dt.to_period('M').apply(lambda r: r.start_time)
    elif anchor == 'Quarter':
        df['anchor'] = df['datetime'].dt.to_period('Q').apply(lambda r: r.start_time)
    elif anchor == 'Year':
        df['anchor'] = df['datetime'].dt.to_period('Y').apply(lambda r: r.start_time)
    elif anchor == 'Decade':
        df['anchor'] = (df['datetime'].dt.year // 10 * 10).astype(str) + '-01-01'
        df['anchor'] = pd.to_datetime(df['anchor'])
    elif anchor == 'Century':
        df['anchor'] = (df['datetime'].dt.year // 100 * 100).astype(str) + '-01-01'
        df['anchor'] = pd.to_datetime(df['anchor'])
    elif isinstance(anchor, pd._libs.tslibs.timestamps.Timestamp) and column_name == "Anchor_Hi":
        # Apply the start_datetime if provided
        df = df[df['datetime'] >= anchor]
        df['anchor'] = anchor
        df['TPV'] = df['high'] * df['volume']
    elif isinstance(anchor, pd._libs.tslibs.timestamps.Timestamp) and column_name == "Anchor_Lo":
        # Apply the start_datetime if provided
        df = df[df['datetime'] >= anchor]
        df['anchor'] = anchor
        df['TPV'] = df['low'] * df['volume']

    else:
        raise ValueError("Unsupported anchor period.")
    
    # Group by date and anchor
    df['Cumulative_TPV'] = df.groupby(['date', 'anchor'])['TPV'].cumsum()
    df['Cumulative_Volume'] = df.groupby(['date', 'anchor'])['volume'].cumsum()
    df[column_name] = df['Cumulative_TPV'] / df['Cumulative_Volume']
    
    return df

def calculate_Anchor_Hi(df):

    df = df.copy()

    anchor = pd.to_datetime('2024-10-18 00:00:00')
    df = calculate_vwap(df, anchor, 'Anchor_Hi')
    length_df = -len(df)

    hi_ = df['high'].iloc[-len(df)]

    anchor_hi_values = []
    anchor_hi_values.append(df['Anchor_Hi'].iloc[-len(df)])

    conditions_li = []
    conditions_li.append(True)
    
    anchoredHi_highlight_li = []
    anchoredHi_highlight_li.append(df['Anchor_Hi'].iloc[-len(df)])

    for i in range(-len(df)+1, 0):

        condition_1 = df['Py_rising_'].iloc[i]
        condition_2 = df['Py_c_'].iloc[i] > hi_
        condition_3 = (df['Py_c1_'].iloc[i] > df['Anchor_Hi'].iloc[i-1] and df['close'].iloc[i] > df['high'].iloc[i-1] and df['open'].iloc[i] > df['Anchor_Hi'].iloc[i-1] and df['Py_doji_'].iloc[i]) if length_df != i else False
        
        conditions = condition_1 and (condition_2 or condition_3)
        hi_update = conditions

        if str(df['datetime'].iloc[i].time()) == "00:01:00":
            condition_1_lo = df['Py_falling_'].iloc[i]
            condition_2_lo = df['Py_c_'].iloc[i] < df['Anchor_Hi'].iloc[i-1]
            condition_3_lo = (df['Py_c1_'].iloc[i] < df['Anchor_Hi'].iloc[i-1] and df['close'].iloc[i] < df['low'].iloc[i-1] and df['open'].iloc[i] < df['Anchor_Hi'].iloc[i-1] and df['Py_doji_'].iloc[i]) if length_df != i else False
            
            conditions = condition_1_lo and (condition_2_lo or condition_3_lo) if conditions == False else conditions

        if str(df['datetime'].iloc[i].time()) == "00:00:00":
            conditions = True

        if conditions:
            last_anchor_hi = df['high'].iloc[i]
            anchor_hi_values.append(last_anchor_hi)
            conditions_li.append(conditions)
            anchoredHi_highlight_li.append(np.nan)

        else:
            last_anchor_hi = df['Anchor_Hi'].iloc[i]
            anchor_hi_values.append(last_anchor_hi)
            conditions_li.append(conditions)
            anchoredHi_highlight_li.append(df['Anchor_Hi'].iloc[i])

        if conditions:
            anchor = pd.to_datetime(df['datetime'].iloc[i])

            anchor_df1 = calculate_vwap(df, anchor, 'Anchor_Hi')[['Anchor_Hi']]

            hi_ = df['high'].iloc[i] if str(df['datetime'].iloc[i].time()) == "00:01:00" and hi_update else hi_
            hi_ = df['high'].iloc[i] if str(df['datetime'].iloc[i].time()) != "00:01:00" else hi_

            current_len = -len(anchor_df1)
            x = length_df-current_len

            zeros_df = pd.DataFrame({'Anchor_Hi': [0.0] * abs(x)})
            achor_hi2 = pd.concat([zeros_df, anchor_df1], ignore_index=True)

            df.drop(columns=['Anchor_Hi'], inplace=True)
            df['Anchor_Hi'] = achor_hi2['Anchor_Hi']

            anchor_hi_values.pop()
            # anchoredHi_highlight_li.pop()
            last_anchor_hi = df['Anchor_Hi'].iloc[i]
            anchor_hi_values.append(last_anchor_hi)
            # anchoredHi_highlight_li.append(df['Anchor_Hi'].iloc[i])

    df['Anchor_Hi'] = anchor_hi_values

    # --------------------------------

    df['anchoredHi_highlight'] = anchoredHi_highlight_li

    return df

def calculate_Anchor_Lo(df):

    df = df.copy()

    anchor = pd.to_datetime('2024-10-18 00:00:00')
    df = calculate_vwap(df, anchor, 'Anchor_Lo')
    length_df = -len(df)

    lo_ = df['low'].iloc[-len(df)]

    anchor_lo_values = []
    anchor_lo_values.append(df['Anchor_Lo'].iloc[-len(df)])

    conditions_li = []
    conditions_li.append(True)

    anchoredLo_highlight_li = []
    anchoredLo_highlight_li.append(df['Anchor_Lo'].iloc[-len(df)])

    for i in range(-len(df)+1, 0):

        condition_1 = df['Py_falling_'].iloc[i]
        condition_2 = df['Py_c_'].iloc[i] < lo_
        condition_3 = (df['Py_c1_'].iloc[i] < df['Anchor_Lo'].iloc[i-1] and df['close'].iloc[i] < df['low'].iloc[i-1] and df['open'].iloc[i] < df['Anchor_Lo'].iloc[i-1] and df['Py_doji_'].iloc[i]) if length_df != i else False

        conditions = condition_1 and (condition_2 or condition_3)
        lo_update = conditions

        if str(df['datetime'].iloc[i].time()) == "00:01:00":
            condition_1_hi = df['Py_rising_'].iloc[i]
            condition_2_hi = df['Py_c_'].iloc[i] > df['Anchor_Hi'].iloc[i-1]
            condition_3_hi = (df['Py_c1_'].iloc[i] > df['Anchor_Hi'].iloc[i-1] and df['close'].iloc[i] > df['high'].iloc[i-1] and df['open'].iloc[i] > df['Anchor_Hi'].iloc[i-1] and df['Py_doji_'].iloc[i]) if length_df != i else False

            conditions = condition_1_hi and (condition_2_hi or condition_3_hi) if conditions == False else conditions

        if str(df['datetime'].iloc[i].time()) == "00:00:00":
            conditions = True

        if conditions: 
            last_anchor_lo = df['low'].iloc[i]
            anchor_lo_values.append(last_anchor_lo)
            conditions_li.append(conditions)
            anchoredLo_highlight_li.append(np.nan)
        else:
            last_anchor_lo = df['Anchor_Lo'].iloc[i]
            anchor_lo_values.append(last_anchor_lo)
            conditions_li.append(conditions)
            anchoredLo_highlight_li.append(df['Anchor_Lo'].iloc[i])

        if conditions:
            anchor = pd.to_datetime(df['datetime'].iloc[i])

            anchor_df1 = calculate_vwap(df, anchor, 'Anchor_Lo')[['Anchor_Lo']]

            lo_ = df['low'].iloc[i] if str(df['datetime'].iloc[i].time()) == "00:01:00" and lo_update else lo_
            lo_ = df['low'].iloc[i] if str(df['datetime'].iloc[i].time()) != "00:01:00" else lo_

            current_len = -len(anchor_df1)
            x = length_df - current_len

            zeros_df = pd.DataFrame({'Anchor_Lo': [0.0] * abs(x)})
            anchor_lo2 = pd.concat([zeros_df, anchor_df1], ignore_index=True)

            df.drop(columns=['Anchor_Lo'], inplace=True)
            df['Anchor_Lo'] = anchor_lo2['Anchor_Lo']

            anchor_lo_values.pop()
            # anchoredLo_highlight_li.pop()
            last_anchor_lo = df['Anchor_Lo'].iloc[i]
            anchor_lo_values.append(last_anchor_lo)
            # anchoredLo_highlight_li.append(df['Anchor_Lo'].iloc[i])

    df['Anchor_Lo'] = anchor_lo_values

    # --------------------------------

    df['anchoredLo_highlight'] = anchoredLo_highlight_li

    return df

# ==============================================
# ==============================================

df_py = calculate_initial_values(df_py)

df_py = prepare_data(df_py)

# Parameters (These can be modified or set via user input)
# Options: 'Session', 'Week', 'Month', 'Quarter', 'Year', 'Decade', 'Century'

anchor = 'Session'  
df_py = calculate_vwap(df_py, anchor, 'VWAP')

df_py = calculate_Anchor_Hi(df_py)

df_py = calculate_Anchor_Lo(df_py)

df_py['anchoredMean'] = (df_py['Anchor_Hi'] + df_py['Anchor_Lo']) / 2

df_py.drop(columns=['date', 'Typical_Price', 'TPV', 'anchor', 'Cumulative_Volume', 'Cumulative_TPV'], inplace=True)
df_py = df_py[['datetime', 'open', 'high', 'low', 'close', 'volume', 'VWAP', 'Anchor_Hi', 'Anchor_Lo', 'anchoredMean', 'anchoredHi_highlight', 'anchoredLo_highlight']] 

def create_highlight_columns(df):
    # Create a copy of the DataFrame to avoid chain indexing issues
    df = df.copy()
    
    # Create new columns for both high and low
    df['anchoredHi_highlight_1'] = np.nan
    df['anchoredHi_highlight_2'] = np.nan
    df['anchoredLo_highlight_1'] = np.nan
    df['anchoredLo_highlight_2'] = np.nan

    # Function to process a column and distribute values into two new columns alternating on NaN
    def process_column(dataframe, source_col, dest_col1, dest_col2):
        current_col = 2  # Start with column 2 to toggle to 1 first
        prev_is_na = True
        for idx in dataframe.index:
            val = dataframe.loc[idx, source_col]
            if pd.notna(val):
                if prev_is_na:
                    current_col = 3 - current_col  # Toggles between 1 and 2
                dest_col = dest_col1 if current_col == 1 else dest_col2
                dataframe.loc[idx, dest_col] = val
                prev_is_na = False
            else:
                prev_is_na = True

        return dataframe

    # Process the high and low columns
    df = process_column(df, 'anchoredHi_highlight', 'anchoredHi_highlight_1', 'anchoredHi_highlight_2')
    df = process_column(df, 'anchoredLo_highlight', 'anchoredLo_highlight_1', 'anchoredLo_highlight_2')

    # add Anchor_Hi and Anchor_Lo columns values to the highlight columns
    # Using proper loc assignment to avoid the SettingWithCopyWarning
    for i in range(len(df) - 1):
        if pd.isna(df.loc[df.index[i], 'anchoredHi_highlight_1']) and pd.notna(df.loc[df.index[i+1], 'anchoredHi_highlight_1']):
            df.loc[df.index[i], 'anchoredHi_highlight_1'] = df.loc[df.index[i], 'Anchor_Hi']
        
        if pd.isna(df.loc[df.index[i], 'anchoredHi_highlight_2']) and pd.notna(df.loc[df.index[i+1], 'anchoredHi_highlight_2']):  
            df.loc[df.index[i], 'anchoredHi_highlight_2'] = df.loc[df.index[i], 'Anchor_Hi']

    for i in range(len(df) - 1):
        if pd.isna(df.loc[df.index[i], 'anchoredLo_highlight_1']) and pd.notna(df.loc[df.index[i+1], 'anchoredLo_highlight_1']):
            df.loc[df.index[i], 'anchoredLo_highlight_1'] = df.loc[df.index[i], 'Anchor_Lo']
        
        if pd.isna(df.loc[df.index[i], 'anchoredLo_highlight_2']) and pd.notna(df.loc[df.index[i+1], 'anchoredLo_highlight_2']):
            df.loc[df.index[i], 'anchoredLo_highlight_2'] = df.loc[df.index[i], 'Anchor_Lo']

    return df

# Usage
df_py = create_highlight_columns(df_py)

In [ ]:
def plot_with_bollinger_bands_and_signal_t(df, entry_time, exit_time, entry_price, exit_price, 
                                         position, pnl, trade_n, plot_supertrend = False, save_chart = False):

    column_list = ['anchoredHi_highlight_1', 'anchoredHi_highlight_2', 'anchoredLo_highlight_1', 'anchoredLo_highlight_2']

    for column in column_list:

        index_i = -len(df)

        value = df['Anchor_Hi'].iloc[index_i] if column == 'anchoredHi_highlight_1' or column == 'anchoredHi_highlight_2' else df['Anchor_Lo'].iloc[index_i]

        if df[column].isna().all():
            df.iloc[-len(df), df.columns.get_loc(column)] = value

    # print(entry_time, exit_time, entry_price, exit_price, position, pnl, trade_n)
    supertrend_start_time = entry_time

    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    apdict = [
        mpf.make_addplot(df['VWAP'], color='#2962FF', alpha=1),
        mpf.make_addplot(df['Anchor_Hi'], color='#00BCD4', alpha=0.2),
        mpf.make_addplot(df['Anchor_Lo'], color='#00BCD4', alpha=0.2),
        mpf.make_addplot(df['anchoredMean'], color='#00BCD4', alpha=0.5),
        mpf.make_addplot(df['anchoredHi_highlight_1'], color='#00BCD4', alpha=1.0),
        mpf.make_addplot(df['anchoredHi_highlight_2'], color='#00BCD4', alpha=1.0),
        mpf.make_addplot(df['anchoredLo_highlight_1'], color='#00BCD4', alpha=1.0),
        mpf.make_addplot(df['anchoredLo_highlight_2'], color='#00BCD4', alpha=1.0),
    ]

    df['upper_2'] = (df['Anchor_Hi'] + df['anchoredMean']) / 2
    df['upper_1'] = (df['anchoredMean'] + df['upper_2']) / 2
    df['upper_3'] = (df['Anchor_Hi'] + df['upper_2']) / 2

    df['lower_2'] = (df['Anchor_Lo'] + df['anchoredMean']) / 2
    df['lower_1'] = (df['anchoredMean'] + df['lower_2']) / 2
    df['lower_3'] = (df['Anchor_Lo'] + df['lower_2']) / 2

    fill_between=[
             dict(y1=df['anchoredMean'].values, y2=df['Anchor_Hi'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['upper_1'].values, y2=df['Anchor_Hi'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['upper_2'].values, y2=df['Anchor_Hi'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['upper_3'].values, y2=df['Anchor_Hi'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['anchoredMean'].values, y2=df['Anchor_Lo'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['lower_1'].values, y2=df['Anchor_Lo'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['lower_2'].values, y2=df['Anchor_Lo'].values, color="#00BCD4", alpha=0.03),
             dict(y1=df['lower_3'].values, y2=df['Anchor_Lo'].values, color="#00BCD4", alpha=0.03)
         ]

    def create_signal_df(df, signal_time, compare_str):
        signal_df = pd.DataFrame(index=df.index)
        signal_df['Signal'] = np.nan

        if signal_time != None:
            signal_time_index = df.index.indexer_at_time(pd.to_datetime(signal_time).time())
            if len(signal_time_index) > 0:
                for idx in signal_time_index:
                    if compare_str == "Entry":
                        signal_df.iloc[idx] = entry_price
                    if compare_str == "Exit":
                        signal_df.iloc[idx] = exit_price
            else:
                print(f"No data at time {signal_time}")
        return signal_df

    if entry_time != None:
        entry_df = create_signal_df(df, entry_time, "Entry")
        apdict.append(mpf.make_addplot(entry_df['Signal'], type='scatter', markersize=70, secondary_y='auto', color='g'))

    if plot_supertrend == True:
        exit_time = exit_time - pd.Timedelta(minutes=1)
        exit_time = exit_time.strftime('%Y-%m-%d %H:%M:%S')
        if exit_time != None:
            exit_df = create_signal_df(df, exit_time, "Exit")
            apdict.append(mpf.make_addplot(exit_df['Signal'], type='scatter', markersize=70, secondary_y='auto', color='#ab0000'))
    else:
        if exit_time != None:
            exit_df = create_signal_df(df, exit_time, "Exit")
            apdict.append(mpf.make_addplot(exit_df['Signal'], type='scatter', markersize=70, secondary_y='auto', color='#ab0000'))

    # Plotting Supertrend
    if plot_supertrend == True:
        print(supertrend_start_time, exit_time)
        # if entry_time <= exit_time:
        exit_time = pd.to_datetime(exit_time)
        exit_time = exit_time + pd.Timedelta(minutes=2)

        if position == "Short":
            exit_time = pd.to_datetime(exit_time)
            exit_time = exit_time - pd.Timedelta(minutes=1)
            mask = (df.index < entry_time) | (df.index > exit_time)
            df.loc[mask, 'tsl_upperband'] = None

            apdict.append(mpf.make_addplot(df['tsl_upperband'], secondary_y='auto', color='b'))

        if position == "Long":
            exit_time = pd.to_datetime(exit_time)
            exit_time = exit_time - pd.Timedelta(minutes=1)
            mask = (df.index < supertrend_start_time) | (df.index > exit_time)
            df.loc[mask, 'tsl_lowerband'] = None

            apdict.append(mpf.make_addplot(df['tsl_lowerband'], secondary_y='auto', color='b'))

    # ----- Plotting Chart as output -----
    style = mpf.make_mpf_style(base_mpf_style='yahoo', facecolor='white')

    mpf.plot(df, figratio=(15, 5.7), type='candle', style=style, figscale=1.25, addplot=apdict, 
            scale_width_adjustment=dict(lines=0.70), fill_between = fill_between,
                   returnfig=True)

    # ----- Saving Chart in folder -----
    if save_chart == True:
        pnl_str = str(pnl).replace('.', '_')
        directory = r'C:\Users\LENOVO\Desktop\Visual Studio\Jupyter_Notebook\Bollinger_Bands\Backtesting\output_charts'

        mpf.plot(df, figratio=(15, 5.7), type='candle', style='yahoo', figscale=1.25, addplot=apdict, 
                scale_width_adjustment=dict(lines=0.70), fill_between = fill_between, savefig=f'{directory}\\{pnl_str} {trade_n}.png')

# Select the first 100 rows from bn_df_for_plot
bn_df_for_plot = df_py.iloc[-1440:-1340].copy()

# Set entry and exit times and prices
bn_entry_time_CE = bn_df_for_plot['datetime'].iloc[20]
bn_exit_time_CE = bn_df_for_plot['datetime'].iloc[21]
bn_entry_CE = bn_df_for_plot['close'].iloc[20]
bn_exit_CE = bn_df_for_plot['close'].iloc[21]

plot_with_bollinger_bands_and_signal_t(bn_df_for_plot, bn_entry_time_CE, bn_exit_time_CE, bn_entry_CE, bn_exit_CE, "Long", pnl=1, trade_n=1)